In [1]:
import sys
sys.path.append(r'C:\Users\Administrator\PycharmProjects\backtester')

import logging
logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

from backtester.core.context import Context
from datetime import timedelta
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from backtester.core.portfolio import OptionsPortfolio
from backtester.core.option_pricer import OPTION_TYPE_CALL, OPTION_TYPE_PUT
from backtester.strategies.straddle1 import StrategyStraddle1

def _groupbyInterpFunc(data):
    return interp1d(data['DaysToMaturity'], data['Volatility'], bounds_error=False, fill_value='extrapolate', kind='cubic')

def getMktData():
    filaname = r'C:\Users\Administrator\PycharmProjects\backtester\data\spx_vols.txt'

    df = pd.read_csv(filaname, parse_dates=['Date', 'Maturity']) # dtype={'Date': np.datetime64, 'Maturity': np.datetime64})
    df['Maturity'] = pd.to_datetime(df['Maturity'], format='%Y%m%d')
    df['DaysToMaturity'] = (df['Maturity'] - df['Date']).astype('timedelta64[D]') #TODO: check whether using day instead of business day to interpolate is correct?

    dfGroupedByInterp = pd.DataFrame(df.groupby(['Date', 'Spot']).apply(_groupbyInterpFunc))
    interpFuncDict = dfGroupedByInterp.to_dict('index')
    daysToMaturity = np.array(range(1,91))
    interpDict = {k: v[0](daysToMaturity) for k,v in interpFuncDict.items()}
    listOfDfs = [pd.DataFrame({'Date': k[0], 'Spot': k[1], 'DaysToMaturity':daysToMaturity, 'Volatility':v}) for k,v in interpDict.items()]
    dfInterpolatedVols = pd.concat(listOfDfs)
    dfInterpolatedVols['Maturity'] = dfInterpolatedVols['Date'] + pd.to_timedelta(dfInterpolatedVols['DaysToMaturity'],                                                                                  unit='d')
    dfInterpolatedVols['Stock'] = '.SPY'

    return dfInterpolatedVols




In [ ]:
mktdata = getMktData()
context = Context(mktdata, balance=10000)

strategy = StrategyStraddle1(context)
strategy.run()

INFO:backtester.core.strategy:2013-02-05 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.strategy:2013-02-06 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.strategy:2013-02-07 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.strategy:2013-02-08 00:00:00: Balance: 10000, Options: 0.0, Stock: 0, ref_notional: 10000.0
INFO:backtester.core.portfolio:2013-02-08 00:00:00: trading option: -29.6150928417884 of 2013-03-15 00:00:00 1517.93 call on .SPY @ 21.841628085697607
INFO:backtester.core.portfolio:2013-02-08 00:00:00: trading option: -29.6150928417884 of 2013-03-15 00:00:00 1517.93 put on .SPY @ 20.366579961389334
INFO:backtester.core.portfolio:2013-02-08 00:00:00: trading stock: -1.0705364378459823 on .SPY @ 1517.93
INFO:backtester.core.portfolio:2013-02-11 00:00:00: futures pnl adjustment: 
  Stock    Amount PreviousSpot     Spot       Date       PnL
0  .

INFO:backtester.core.strategy:2013-03-04 00:00:00: Balance: 16479.884111289262, Options: -3985.671532586576, Stock: 0, ref_notional: 12494.212578702685
INFO:backtester.core.portfolio:2013-03-04 00:00:00: trading stock: -18.028794220131438 on .SPY @ 1525.2
INFO:backtester.core.portfolio:2013-03-05 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -21.999566        1525.2  1539.79 2013-03-05 -320.973663
INFO:backtester.core.strategy:2013-03-05 00:00:00: Balance: 16158.910447911136, Options: -4320.054162098436, Stock: 0, ref_notional: 11838.8562858127
INFO:backtester.core.portfolio:2013-03-05 00:00:00: trading stock: -37.764645464270146 on .SPY @ 1539.79
INFO:backtester.core.portfolio:2013-03-06 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date        PnL
0  .SPY -59.764211       1539.79  1541.46 2013-03-06 -99.806233
INFO:backtester.core.strategy:2013-03-06 00:00:00: Balance: 16059.10421528796

INFO:backtester.core.portfolio:2013-03-22 00:00:00: trading stock: 10.8594747262256 on .SPY @ 1556.89
INFO:backtester.core.portfolio:2013-03-25 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date       PnL
0  .SPY -50.161962       1556.89  1551.69 2013-03-25  260.8422
INFO:backtester.core.strategy:2013-03-25 00:00:00: Balance: 16855.883986345278, Options: -5683.523128243243, Stock: 0, ref_notional: 11172.360858102034
INFO:backtester.core.portfolio:2013-03-25 00:00:00: trading stock: 4.400748441163046 on .SPY @ 1551.69
INFO:backtester.core.portfolio:2013-03-26 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -45.761213       1551.69  1563.77 2013-03-26 -552.795454
INFO:backtester.core.strategy:2013-03-26 00:00:00: Balance: 16303.088532574291, Options: -5922.4648563071905, Stock: 0, ref_notional: 10380.6236762671
INFO:backtester.core.portfolio:2013-03-26 00:00:00: trading stock: -29.26257328832

INFO:backtester.core.portfolio:2013-04-12 00:00:00: futures pnl adjustment: 
  Stock      Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -102.740966       1593.37  1588.85 2013-04-12  464.389168
INFO:backtester.core.strategy:2013-04-12 00:00:00: Balance: 15130.105902230502, Options: -3604.892159643362, Stock: 0, ref_notional: 11525.21374258714
INFO:backtester.core.portfolio:2013-04-12 00:00:00: trading option: -35.00209533667749 of 2013-05-17 00:00:00 1588.85 call on .SPY @ 21.35148910651185
INFO:backtester.core.portfolio:2013-04-12 00:00:00: trading option: -35.00209533667749 of 2013-05-17 00:00:00 1588.85 put on .SPY @ 19.807524488705212
INFO:backtester.core.portfolio:2013-04-12 00:00:00: trading stock: 34.58967522900788 on .SPY @ 1588.85
INFO:backtester.core.portfolio:2013-04-15 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date          PnL
0  .SPY -68.151291       1588.85  1552.36 2013-04-15  2486.840616
INFO:backtester.core.strat

INFO:backtester.core.strategy:2013-05-02 00:00:00: Balance: 24095.607794339998, Options: -6951.46964991494, Stock: 0, ref_notional: 17144.13814442506
INFO:backtester.core.portfolio:2013-05-02 00:00:00: trading stock: -40.23176618363801 on .SPY @ 1597.59
INFO:backtester.core.portfolio:2013-05-03 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date          PnL
0  .SPY -81.672411       1597.59  1614.42 2013-05-03 -1374.546679
INFO:backtester.core.strategy:2013-05-03 00:00:00: Balance: 22721.061115157958, Options: -8316.95933804442, Stock: 0, ref_notional: 14404.101777113538
INFO:backtester.core.portfolio:2013-05-03 00:00:00: trading option: -40.455902429651005 of 2013-06-07 00:00:00 1614.42 call on .SPY @ 23.037187726506318
INFO:backtester.core.portfolio:2013-05-03 00:00:00: trading option: -40.455902429651005 of 2013-06-07 00:00:00 1614.42 put on .SPY @ 21.468375467149144
INFO:backtester.core.portfolio:2013-05-03 00:00:00: trading stock: -38.3480059515

...

INFO:backtester.core.strategy:2013-10-30 00:00:00: Balance: 18655.673052038863, Options: -9008.736863476257, Stock: 0, ref_notional: 9646.936188562606
INFO:backtester.core.portfolio:2013-10-30 00:00:00: trading stock: 9.481747685843189 on .SPY @ 1763.31
INFO:backtester.core.portfolio:2013-10-31 00:00:00: futures pnl adjustment: 
  Stock      Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -100.711437       1763.31  1756.54 2013-10-31  681.816431
INFO:backtester.core.strategy:2013-10-31 00:00:00: Balance: 19337.489483508187, Options: -8283.2719017696, Stock: 0, ref_notional: 11054.217581738587
INFO:backtester.core.portfolio:2013-10-31 00:00:00: trading stock: 7.311156486542288 on .SPY @ 1756.54
INFO:backtester.core.portfolio:2013-11-01 00:00:00: expiring options
  Stock   Strike   Maturity  PutCall     Amount       Date     Spot  \
0  .SPY  1691.75 2013-11-01        1 -28.248936 2013-11-01  1761.64   
1  .SPY  1691.75 2013-11-01       -1 -28.248936 2013-11-01  1761.64   

 

INFO:backtester.core.portfolio:2013-11-19 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -80.666223       1791.53  1787.87 2013-11-19  295.238376
INFO:backtester.core.strategy:2013-11-19 00:00:00: Balance: 13564.256174400562, Options: -5402.735896415483, Stock: 0, ref_notional: 8161.520277985079
INFO:backtester.core.portfolio:2013-11-19 00:00:00: trading stock: 5.728322314658911 on .SPY @ 1787.87
INFO:backtester.core.portfolio:2013-11-20 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -74.937901       1787.87  1781.37 2013-11-20  487.096355
INFO:backtester.core.strategy:2013-11-20 00:00:00: Balance: 14051.352529168733, Options: -4897.00977228072, Stock: 0, ref_notional: 9154.342756888012
INFO:backtester.core.portfolio:2013-11-20 00:00:00: trading stock: 10.21170653829276 on .SPY @ 1781.37
INFO:backtester.core.portfolio:2013-11-21 00:00:00: futures pnl adjustment: 
  

INFO:backtester.core.strategy:2013-12-09 00:00:00: Balance: 10295.790746188182, Options: -3405.0922700465003, Stock: 0, ref_notional: 6890.6984761416825
INFO:backtester.core.portfolio:2013-12-09 00:00:00: trading stock: -7.897366384763057 on .SPY @ 1808.37
INFO:backtester.core.portfolio:2013-12-10 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date        PnL
0  .SPY -35.284061       1808.37  1802.62 2013-12-10  202.88335
INFO:backtester.core.strategy:2013-12-10 00:00:00: Balance: 10498.674096079756, Options: -3280.2779688626188, Stock: 0, ref_notional: 7218.396127217137
INFO:backtester.core.portfolio:2013-12-10 00:00:00: trading stock: 7.054832269000066 on .SPY @ 1802.62
INFO:backtester.core.portfolio:2013-12-11 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -28.229229       1802.62  1782.22 2013-12-11  575.876263
INFO:backtester.core.strategy:2013-12-11 00:00:00: Balance: 11074.5503591466

INFO:backtester.core.portfolio:2013-12-31 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -50.553332       1841.07  1848.36 2013-12-31 -368.533793
INFO:backtester.core.strategy:2013-12-31 00:00:00: Balance: 12618.273201725598, Options: -4899.585823308116, Stock: 0, ref_notional: 7718.687378417482
INFO:backtester.core.portfolio:2013-12-31 00:00:00: trading stock: -5.901528962767998 on .SPY @ 1848.36
INFO:backtester.core.portfolio:2014-01-02 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date        PnL
0  .SPY -56.454861       1848.36  1831.98 2014-01-02  924.73063
INFO:backtester.core.strategy:2014-01-02 00:00:00: Balance: 13543.003831311562, Options: -4045.295139093168, Stock: 0, ref_notional: 9497.708692218393
INFO:backtester.core.portfolio:2014-01-02 00:00:00: trading stock: 14.342404075821648 on .SPY @ 1831.98
INFO:backtester.core.portfolio:2014-01-03 00:00:00: expiring options
  Stock  

INFO:backtester.core.strategy:2014-01-21 00:00:00: Balance: 15953.86054066599, Options: -4151.499299987973, Stock: 0, ref_notional: 11802.361240678016
INFO:backtester.core.portfolio:2014-01-21 00:00:00: trading stock: -17.39230878424167 on .SPY @ 1843.8
INFO:backtester.core.portfolio:2014-01-22 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date        PnL
0  .SPY -35.563702        1843.8  1844.86 2014-01-22 -37.697525
INFO:backtester.core.strategy:2014-01-22 00:00:00: Balance: 15916.163016088038, Options: -4130.1959634537925, Stock: 0, ref_notional: 11785.967052634245
INFO:backtester.core.portfolio:2014-01-22 00:00:00: trading stock: -3.201864723656854 on .SPY @ 1844.86
INFO:backtester.core.portfolio:2014-01-23 00:00:00: futures pnl adjustment: 
  Stock     Amount  PreviousSpot     Spot       Date         PnL
0  .SPY -38.765567       1844.86  1828.46 2014-01-23  635.755301
INFO:backtester.core.strategy:2014-01-23 00:00:00: Balance: 16551.91831744133

In [6]:
df1 = pd.concat(strategy.optionsPortfolio.dfPositionsHist)
df2= pd.concat(strategy.stockPortfolio.dfPositionsHist)
df3 = pd.DataFrame(strategy.notionalDF, columns=['Date', 'Balance', 'OptionsNotional'])
df = pd.merge(df1, df2, how='outer', on=['Date']).join(df3, on=['Date'])
df.to_csv('c:/temp/results2,csv')
#from IPython.display import display, HTML
#display(HTML(df.to_html()))

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.
